# カタカナ15文字を自動識別するモデルの構築
# 課題投稿用Notebook
* これは課題投稿用ファイルです。
* このsubmit_katakana.ipynbファイルを編集し、学習済みモデルなどの必要ファイルを同じフォルダにおき、それをzipしたものを投稿してください。

In [8]:
import keras
from PIL import Image
import numpy as np
import glob
import os,sys
import util
from common.config import GPU
if GPU:
    import cupy as np
else:
    import numpy as np

from dataset.katakana_arg import load_katakana_arg
from common.trainer import Trainer
from common.layers import *
import pickle

class CNNNet:
    '''
    ネットワーク構成は下記の通り
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        affine - relu - dropout - affine - dropout - softmax
    '''

    def __init__(self, input_dim=(1, 28, 28),
                 conv_param_1={'filter_num': 16, 'filter_size': 3, 'pad': 1, 'stride': 1},
                 conv_param_2={'filter_num': 16, 'filter_size': 3, 'pad': 1, 'stride': 1},
                 conv_param_3={'filter_num': 32, 'filter_size': 3, 'pad': 1, 'stride': 1},
                 conv_param_4={'filter_num': 32, 'filter_size': 3, 'pad': 2, 'stride': 1},
                 conv_param_5={'filter_num': 64, 'filter_size': 3, 'pad': 1, 'stride': 1},
                 conv_param_6={'filter_num': 64, 'filter_size': 3, 'pad': 1, 'stride': 1},
                 hidden_size=50, output_size=15):
        # 重みの初期化===========
        # 各層のニューロンひとつあたりが、前層のニューロンといくつのつながりがあるか
        pre_node_nums = np.array([1 * 3 * 3, 16 * 3 * 3, 16 * 3 * 3, 32 * 3 * 3, 32 * 3 * 3, 64 * 3 * 3, 64 * 4 * 4, hidden_size])
        wight_init_scales = np.sqrt(2.0 / pre_node_nums)  # ReLUを使う場合に推奨される初期値

        self.params = {}
        pre_channel_num = input_dim[0]
        for idx, conv_param in enumerate(
                [conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params['W' + str(idx + 1)] = wight_init_scales[idx] * np.random.randn(conv_param['filter_num'],
                                                                                       pre_channel_num,
                                                                                       conv_param['filter_size'],
                                                                                       conv_param['filter_size'])
            self.params['b' + str(idx + 1)] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
        self.params['W7'] = wight_init_scales[6] * np.random.randn(64 * 4 * 4, hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = wight_init_scales[7] * np.random.randn(hidden_size, output_size)
        self.params['b8'] = np.zeros(output_size)

        # レイヤの生成===========
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'],
                                       conv_param_1['stride'], conv_param_1['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W2'], self.params['b2'],
                                       conv_param_2['stride'], conv_param_2['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W3'], self.params['b3'],
                                       conv_param_3['stride'], conv_param_3['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W4'], self.params['b4'],
                                       conv_param_4['stride'], conv_param_4['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W5'], self.params['b5'],
                                       conv_param_5['stride'], conv_param_5['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W6'], self.params['b6'],
                                       conv_param_6['stride'], conv_param_6['pad']))
        self.layers.append(Relu())
        #self.layers.append(BatchNormalization(1. ,0.)
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(Relu())
        #self.layers.append(BatchNormalization(1. ,0.)
        self.layers.append(Dropout(0.5))
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        #self.layers.append(BatchNormalization(1. ,0.)
        self.layers.append(Dropout(0.5))

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x

    def loss(self, x, t, train_flg=True):
        y = self.predict(x, train_flg)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1: t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i * batch_size:(i + 1) * batch_size]
            tt = t[i * batch_size:(i + 1) * batch_size]
            y = self.predict(tx, train_flg=False)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]

    def accuracy_and_loss(self, x, t, batch_size=100):
        if t.ndim != 1:
            t = np.argmax(t, axis=1)

        acc = 0.0
        loss = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i * batch_size:(i + 1) * batch_size]
            tt = t[i * batch_size:(i + 1) * batch_size]
            y = self.predict(tx, train_flg=False)
            loss += self.last_layer.forward(y, tt)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0], loss / (int(x.shape[0] / batch_size))

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)

        # 設定
        grads = {}
        counter = 1
        for i, layer in enumerate(self.layers):
            if layer.hasWeight: #　今のところWeightをもっているものはBiasがあるので
                grads['W' + str(counter)] = layer.dW
                grads['b' + str(counter)] = layer.db
                counter += 1

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            if GPU:
                import numpy
                val = np.asnumpy(val) # 実際は、cupy.asnumpy()
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            if GPU:
                import numpy
                val = np.array(val) # 実際はcupy.array()
            self.params[key] = val

        counter = 1
        for i, layer in enumerate(self.layers):
            if layer.hasWeight: #　今のところWeightをもっているものはBiasがあるので
                layer.W = self.params['W' + str(counter)]
                layer.b = self.params['b' + str(counter)]
                counter += 1
    
    
def makedataset():
    """
    データセットをつくる関数です。
    自由に編集してください。
    """
    # 次の行は変更しないこと
    test_data= util.loaddata()
    
    # 以下は自由に編集しても構いません
    # 正規化
    test_data = test_data / test_data.max()
    test_data = test_data.astype('float32')

    return test_data


def func_predict(test_data, test_label):
    """
    予測する関数
    data : 画像データ
    return loss, accuracy
    引数とreturn以外は、自由に編集してください    
    """
    
    # ここを自由に編集してください
    # 以下はTensoFlowを用いた場合の例
    model = CNNNet()
    model.load_params('params03.pkl')
    accuracy, loss = model.accuracy_and_loss(test_data, test_label)
    
    return loss, accuracy # 編集不可


def main():
    """
    編集しないでください。
    """
    # テスト用データをつくる
    test_data = makedataset()

    # 予測し精度を算出する
    util.accuracy(func_predict, test_data)
    
    return


if __name__=="__main__":
    main()

0
7.819837041713713e-05 1.0
Test loss: 7.819837041713713e-05
Test accuracy: 1.0
